# Make_PathInfo

In [ ]:
#make_pathInfo_1

from modules.dataset import *
from modules.patch import *
from modules.slide_info import *

import openslide, argparse, pickle, os
import numpy as np


def main():
   
    parser = argparse.ArgumentParser(description="variable file select")
    parser.add_argument("--tsv", default='./data/codipai.tsv', type=str, help="variable tsv file")
    args, unknown = parser.parse_known_args()
    variableFile = args.tsv

    print("Import file")
    inputValue = {}
    for key, value in np.loadtxt(variableFile, dtype=str, delimiter='\t'):
        inputValue[key] = value
    print("Complete\n")

    print("Match file path")
    pathRootWSIList = []
    pathRootAnnoList = []
    for key, value in inputValue.items():
        value = value.split("\n")[0]
        if "pathRootWSI" in key:
            pathRootWSIList.append(value)
        if "pathRootAnno" in key:
            pathRootAnnoList.append(value)

    pathWSIDic = {}
    for pathRootWSI in pathRootWSIList:
        pathWSIDic = {**pathWSIDic, **ReadFilesSubDir(pathRootWSI, [".svs", ".tiff"])}

    pathAnnoDic = {}
    for pathRootAnno in pathRootAnnoList:
        pathAnnoDic = {**pathAnnoDic, **ReadFilesSubDir(pathRootAnno, [".xml", ".geojson"])}

    intersectID = np.intersect1d(list(pathWSIDic.keys()), list(pathAnnoDic.keys()))
    pathDicSum = {}
    for ID in intersectID:
        pathDicSum[ID] = {"pathWSI" : pathWSIDic[ID], "pathAnno" : pathAnnoDic[ID]}
    print("Complete\n")

    print("Save path pack")
    pickle.dump(pathDicSum, open("./data/PATH/pathMatchWSINAnno_457.dump", "wb"))
    print("Complete\n")
    
    print("Integrity check")
    errorSlide = []
    for ID in pathWSIDic:
        try:
            handleWSI = openslide.OpenSlide(pathWSIDic[ID])
            boundDic = GetBoundDic(handleWSI)
            img = GetWSIThumbnail(handleWSI, handleWSI.level_downsamples[-1], boundDic)
        except:
            print("error in slide %s"%pathWSIDic[ID])
            errorSlide.append(pathWSIDic[ID])
            np.savetxt("./data/PATH/errorSlide_2.txt", errorSlide, fmt="%s")
    print("Complete\n")
    
    return None

if __name__ == "__main__":
    main()

# MakingAnnotation_mask

In [ ]:
#Making_Annotation_mask using svs images

from modules.preprocess import *
from modules.dataset import *
from modules.slide_info import *
from modules.patch import *
# from modules.preprocess import *

import openslide, argparse, pickle, tifffile, cv2, os
import numpy as np


def main():
    
    parser = argparse.ArgumentParser(description="variable file select")
    parser.add_argument("--tsv", default='./data/codipai_anno.tsv', type=str, help="variable tsv file")
    args, unknown = parser.parse_known_args()
    variableFile = args.tsv
    
    # get argments
    print("Import Files")
    inputValue = {}
    for key, value in np.loadtxt(variableFile, dtype=str, delimiter='\t'):
        inputValue[key] = value

    pathDicSum = str(inputValue["pathDicSum"])
    pathRootSave = str(inputValue["pathRootSave"])
    pixelAreaReal = float(inputValue["pixelAreaReal"])
    pixelAreaPreview = float(inputValue["pixelAreaPreview"])
    
    pathDicSum = pickle.load(open(pathDicSum, "rb"))
    print("Complete\n")
    
    print("Make Annotation Dictionary")
    polygonDicTotal = {}

#     for ID in pathDicSum:
#         polygonDicTotal[ID] = OrganizePloygon(pathDicSum[ID]["pathAnno"])
        
#     pickle.dump(polygonDicTotal, open("polygonDicTotal.dump", "wb"))
    print("Complete\n")
    
    print("Save Annotation Mask & List")
    errorList = []
    for ID in pathDicSum:
        print(ID)
        
        pathWSI = pathDicSum[ID]["pathWSI"]
        pathAnno = pathDicSum[ID]["pathAnno"]
        
        if ".xml" in pathAnno:
            typeParse = "aperio_xml"
        elif ".geojson" in pathAnno:
            typeParse = "qupath_geojson"
            
        OS = openslide.OpenSlide(pathWSI)

        boundDic = GetBoundDic(OS)

        pixelTotal = boundDic['w'] * boundDic['h'] / np.array(OS.level_downsamples)**2
        if pixelAreaReal:
            level = np.argmin(np.abs(np.log10(pixelTotal / pixelAreaReal)))
        else:
            level = 0
        ratioDown = OS.level_downsamples[level]
#         for kind in polygonDicTotal[ID]:
#             if bool(kind):
#                 print(kind)
        pathRootSaveTiff = os.path.join(pathRootSave, "mask_457/TP_Necrosis/")
#         pathRootSaveDump = os.path.join(pathRootSave, "dump/")
        pathRootSavePreV = os.path.join(pathRootSave, "original_jpg/")
        CheckNMakeDirectoryTree(os.path.join(pathRootSaveTiff, "a.temp"))
#         CheckNMakeDirectoryTree(os.path.join(pathRootSaveDump, "a.temp"))
        CheckNMakeDirectoryTree(os.path.join(pathRootSavePreV, "a.temp"))
        
        try:

    #         mask = ConvertPolygon2Mask(ratioDown, boundDic, polygonDicTotal[ID][kind])
            mask = MakeAnnotationMask(ratioDown, boundDic, pathAnno, typeParse)

#             tifffile.imwrite(os.path.join(pathRootSaveTiff, ID+".tiff"), mask, bigtiff=True, compression='zlib', planarconfig='separate', tile=(224, 224))

    #             pickle.dump(polygonDicTotal[ID][kind], open(os.path.join(pathRootSaveDump, ID+".dump"), "wb"))

            level = np.argmin(np.abs(np.log10(pixelTotal / pixelAreaPreview)))
#             thumbImage = GetWSIThumbnail(OS, OS.level_downsamples[level], boundDic)
            thumbImage = GetWSIThumbnail(OS, 5, boundDic)
            cv2.imwrite(os.path.join(pathRootSavePreV, ID+"+original.jpg"), cv2.cvtColor(thumbImage, cv2.COLOR_BGR2RGB))

            mask= cv2.resize(mask, dsize=(thumbImage.shape[1], thumbImage.shape[0]))
            thumbImage[mask == 0] = (thumbImage[mask == 0] / 3 * 2).astype(np.uint8)
#             cv2.imwrite(os.path.join(pathRootSavePreV, ID+"+masked.jpg"), cv2.cvtColor(thumbImage, cv2.COLOR_BGR2RGB))
        except:
            print("masking error at %s"%pathWSI)
            errorList.append(pathWSI)
            np.savetxt("errorList.txt", errorList, fmt="%s")
            
            
        print("Complete\n")

    return None

if __name__ == "__main__":
    main()

# Make patches for multiple classes

In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
import tifffile as tiff
import cv2

from glob import glob
from tqdm import tqdm
from patchify import patchify


class_names = ['TP_necrosis']


class_files = {
    class_name: {'class_id': class_id+1,
                 'files': [file.replace('.tiff', '')
                           for file in os.listdir(f'/NAS1/HnE_images/processing/CODIPAI/PA/mask/{class_name}')
                           if 'tiff' in file]}
    for class_id, class_name in enumerate(class_names)
}

# print(class_files)
class CODIPAIDataSetting:
    def __init__(self, my_main_dir: str,
                 main_dir: str,
                 class_files: dict,
                 img_dir_prefix: str='',
                 mask_dir_prefix: str='',
                 img_file_prefix='original',
                 mask_file_prefix=''):
        
        self.my_main_dir = my_main_dir
        self.main_dir = main_dir
        self.mapping_table = {}

        self.img_dir_prefix = img_dir_prefix
        self.mask_dir_prefix = mask_dir_prefix
        self.img_file_prefix = img_file_prefix
        self.mask_file_prefix = mask_file_prefix

        self.length = 0
        self.total_task = 0

        self.classes_in_file = {}

        for class_name, class_content in class_files.items():            
            class_id = class_content['class_id']
            self.mapping_table[class_name] = class_id-1

            for file in class_content['files']:
                if self.classes_in_file.get(file) is None:
                    self.classes_in_file[file] = [class_name]
                else:
                    self.classes_in_file[file].append(class_name)

                self.length += 1

    def to_numpy(self):
        os.makedirs(os.path.join(self.my_main_dir, 'data2', 'image'), exist_ok=True)
        os.makedirs(os.path.join(self.my_main_dir, 'data2', 'mask'), exist_ok=True)

        for file_name, classes in tqdm(self.classes_in_file.items(), total=len(self.classes_in_file.keys())):
            img_dir = glob(os.path.join(self.main_dir, self.img_dir_prefix, classes[0], f'*{file_name}*{self.img_file_prefix}*'))
            print(img_dir)
            # 반드시 이미지 하나만
            assert len(img_dir) == 1

            image = cv2.imread(img_dir[0])
            image = image.astype(np.float32)
            image = image / 255.0

            image_shape = image.shape

            image_patches = patchify(image, (512, 512, 3), step=512).squeeze(axis=2)
            image_patches = image_patches.reshape(-1, 512, 512, 3)
            
            masks = np.zeros(shape=(image_patches.shape[0], 6, 512, 512, 1))
#             (image_patches.shape[0], 512, 512) instead of (image_patches.shape[0], 6, 512, 512, 1).


            for class_name in classes:
                mask_dir = glob(os.path.join(self.main_dir, self.mask_dir_prefix, class_name, f'*{file_name}*{self.mask_file_prefix}*'))
                class_id = self.mapping_table[class_name]

                # 반드시 이미지 하나만
                assert len(mask_dir) == 1

                mask = tiff.imread(mask_dir[0])
                mask = np.where(mask != 0, 1, 0)
                mask = mask.astype(np.uint8)
                mask = cv2.resize(mask, image_shape[::-1][1:])
                mask = mask.astype(np.float32)
                mask = np.expand_dims(mask, axis=-1)

                mask_patches = patchify(mask, (512, 512, 1), step=512).squeeze(axis=2).reshape(-1, 512, 512, 1)

                for patch_num, (image, mask) in enumerate(zip(image_patches, mask_patches)):
#                     print(patch_num)
#                     print(class_id)
                    masks[patch_num, class_id] = mask


            for idx, (img, mask) in enumerate(zip(image_patches, masks)):
                f_name = f'{file_name}_{idx:03}.npy'

#                 np.save(os.path.join(self.my_main_dir, 'data/val_test/test/image', f_name), img)
#                 np.save(os.path.join(self.my_main_dir, 'data/val_test/test/mask', f_name), mask)
                np.save(os.path.join(self.my_main_dir, 'data2/image', f_name), img)
                np.save(os.path.join(self.my_main_dir, 'data2/mask', f_name), mask)
                


CODIPAIDataSetting("./data/", "/NAS1/HnE_images/processing/CODIPAI/PA", class_files,
                img_dir_prefix='image',
                mask_dir_prefix='mask',
                img_file_prefix='original', mask_file_prefix='').to_numpy()

# Make patches for one class

In [1]:
# Making pathes

import os
from matplotlib import pyplot as plt
import numpy as np
import tifffile as tiff
import cv2

from glob import glob
from tqdm import tqdm
from patchify import patchify


# class_names = ['TP_Necrosis']

class_names = ['exp']

class_files = {
    class_name: {'class_id': class_id+1,
                 'files': [file.replace('.tiff', '')
                           for file in os.listdir(f'./data/MASK/Annotation/mask_3/{class_name}')
                           if 'tiff' in file]}
    for class_id, class_name in enumerate(class_names)
}

class CODIPAIDataSetting:
    def __init__(self, main_dir: str,
                 class_files: dict,
                 img_dir_prefix: str='',
                 mask_dir_prefix: str='',
                 img_file_prefix='original',
                 mask_file_prefix=''):

        self.main_dir = main_dir
        self.mapping_table = {}

        self.img_dir_prefix = img_dir_prefix
        self.mask_dir_prefix = mask_dir_prefix
        self.img_file_prefix = img_file_prefix
        self.mask_file_prefix = mask_file_prefix

        self.length = 0
        self.total_task = 0

        self.classes_in_file = {}

        for class_name, class_content in class_files.items():            
            class_id = class_content['class_id']
            self.mapping_table[class_name] = class_id-1

            for file in class_content['files']:
                if self.classes_in_file.get(file) is None:
                    self.classes_in_file[file] = [class_name]
                else:
                    self.classes_in_file[file].append(class_name)

                self.length += 1

    def to_numpy(self):
#         os.makedirs(os.path.join(self.main_dir, 'data2', 'image'), exist_ok=True)
#         os.makedirs(os.path.join(self.main_dir, 'data2', 'mask'), exist_ok=True)

        for file_name, classes in tqdm(self.classes_in_file.items(), total=len(self.classes_in_file.keys())):
            img_dir = glob(os.path.join(self.main_dir, self.img_dir_prefix, classes[0], f'*{file_name}*{self.img_file_prefix}*'))
            # 반드시 이미지 하나만
            assert len(img_dir) == 1
            
#             path_img ='./data/MASK/Annotation/preview_org/TP_Necrosis/CODIPAI-BRTU-SS-00142-S-TP-01.jpg'
#             print(path_img)

            image = cv2.imread(img_dir[0])
#             image = cv2.imread(path_img)
            image = image.astype(np.float32)
            image = image / 255.0

            image_shape = image.shape

            image_patches = patchify(image, (512, 512, 3), step=512).squeeze(axis=2)
            image_patches = image_patches.reshape(-1, 512, 512, 3)

            masks = np.zeros(shape=(image_patches.shape[0], 512, 512, 1))
#             print('5: ', masks.shape)

            for class_name in classes:
                mask_dir = glob(os.path.join(self.main_dir, self.mask_dir_prefix, class_name, f'*{file_name}*{self.mask_file_prefix}*'))
                class_id = self.mapping_table[class_name]
  

                # 반드시 이미지 하나만
                assert len(mask_dir) == 1
        
#                 path_mask = './data/MASK/Annotation/mask_3/TP_Necrosis/CODIPAI-BRTU-SS-00142-S-TP-01.tiff'

                mask = tiff.imread(mask_dir[0])
#                 mask = tiff.imread(path_mask)
                mask = np.where(mask != 0, 1, 0)
                mask = mask.astype(np.uint8)
                mask = cv2.resize(mask, image_shape[::-1][1:])
                mask = mask.astype(np.float32)
                mask = np.expand_dims(mask, axis=-1)

                mask_patches = patchify(mask, (512, 512, 1), step=512).squeeze(axis=2).reshape(-1, 512, 512, 1)

                for patch_num, (image, mask) in enumerate(zip(image_patches, mask_patches)):
                    masks[patch_num] = mask


            for idx, (img, mask) in enumerate(zip(image_patches, masks)):

                f_name = f'{file_name}_{idx:03}.npy'

                np.save(os.path.join(self.main_dir, 'data2/test/image', f_name), img)
                np.save(os.path.join(self.main_dir, 'data2/test/mask', f_name), mask)
                


CODIPAIDataSetting("./data/", class_files,
                img_dir_prefix='MASK/Annotation/preview_3',
                mask_dir_prefix='MASK/Annotation/mask_3',
                img_file_prefix='original', mask_file_prefix='').to_numpy()